# Requirements

In [ ]:
!pip install transformers==4.18.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 48.5 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece==0.1.96 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.3 MB/s eta 0:00:00


In [ ]:
!pip install pytorch_lightning==0.8.1 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.1/293.1 kB 6.8 MB/s eta 0:00:00


In [ ]:
!pip install editdistance==0.6.0 -q

  Preparing metadata (setup.py) ... done


In [ ]:
# !pip install numpy==1.22.3 -q
# !pip install scikit-learn==0.24.2 -q
# !pip install tqdm==4.64.0 -q

In [ ]:
# 파이썬 3.10 이상인 경우 실행하여 collection 라이브러리 import 오류 수정
!sed -i '1s/.*/from collections.abc import Mapping, Sequence/' '/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/apply_func.py'

In [ ]:
from datetime import datetime

import random
import torch
import numpy as np
# 랜덤시드 고정
SEED = 1398

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)
random.seed(SEED)

# Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/Colab Notebooks/GW"

/content/drive/MyDrive/Colab Notebooks/GW


# 모델

In [ ]:
# constants.py
T5_ORI_LEN = 64100 #T5_ORI_LEN = 32100

TAG_TO_WORD = {'POS': 'positive', 'NEG': 'negative', 'NEU': 'neutral'}
TAG_WORD_LIST = ['positive', 'negative', 'neutral']

# annotation-special
NONE_TOKEN = "[none]"
AND_TOKEN = "[and]"
ASPECT_TOKEN = "<aspect>"
OPINION_TOKEN = "<opinion>"
SEP_TOKEN = "<sep>"
TAG_TO_SPECIAL = {"POS": ("<pos>", "</pos>"), "NEG": ("<neg>", "</neg>"), "NEU": ("<neu>", "</neu>")}
SPECIAL_TO_TAG = {"<pos>": "positive", "<neg>": "negative", "<neu>": "neutral"}


# TARGET_TEST_COUNT_DICT = {
#     "blog": 213,
#     "youtube": 72,
#     "news": 63,
#     "merge": 134,
#     "mix": 134,
#     "total": 348
# }

TARGET_TEST_COUNT_DICT = {
    "total": 348
}

ASTE_TRANSFER_PAIRS = {
    "total": ["total"]
}

STOP_WORDS = []

In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
import logging
logger = logging.getLogger("BGCA")

In [ ]:
#data_utils.py
# This file contains all data loading and transformation functions

import random
import re

from torch.utils.data import Dataset


senttag2word = {'POS': 'positive', 'NEG': 'negative', 'NEU': 'neutral'}


def filter_invalid(inputs, outputs):
    new_inputs, new_outputs = [], []
    rm_num = 0
    for idx in range(len(inputs)):
        valid = True
        aps = re.findall("(<\w+>)(.*?)(?=<\w+>|$)", outputs[idx])
        for ap in aps:
            for ele in ap:
                # some element is missing
                if len(ele) == 0:
                    valid = False
                    break
            if valid is False:
                break
        if valid:
            new_inputs.append(inputs[idx])
            new_outputs.append(outputs[idx])
        else:
            rm_num += 1
            print(f"Output is invalid: {outputs[idx]}")
    print(f"Filterd out {rm_num} invalid samples")
    return new_inputs, new_outputs


def filter_none(inputs, outputs, ratio=0):
    if ratio > 0:
        new_inputs = []
        new_outputs = []
        rm_num = 0
        none_num = 0
        import random
        for idx in range(len(inputs)):
            if outputs[idx].strip() == NONE_TOKEN:
                none_num += 1
                if random.random() < ratio:
                    rm_num += 1
                    continue
            new_inputs.append(inputs[idx])
            new_outputs.append(outputs[idx])
        print(f"Filtered out {rm_num} [none] samples out of {none_num} [none]")
        return new_inputs, new_outputs
    else:
        return inputs, outputs


def normalize_augment(args, inputs, outputs):
    new_inputs = []
    new_outputs = []
    rm_num = 0
    for idx in range(len(inputs)):
        new_inputs.append(inputs[idx])
        new_outputs.append(outputs[idx])
    return new_inputs, new_outputs


def read_line_examples_from_file(data_path):
    """
    Read data from file, each line is: sent####labels
    Return List[List[word]], List[Tuple]
    """
    sents, labels = [], []
    with open(data_path, 'r', encoding='UTF-8') as fp:
        words, labels = [], []
        for line in fp:
            line = line.strip()
            if line != '':
                words, tuples = line.split('####')
                sents.append(words.split())
                labels.append(eval(tuples))
    print(f"{data_path.split('/')[-1]}\tTotal examples = {len(sents)} ")
    return sents, labels


class ABSADataset(Dataset):
    def __init__(self, args, tokenizer, inputs=None, targets=None, dataset_list=None, name=None):
        self.args = args
        self.tokenizer = tokenizer
        self.inputs, self.targets = inputs or [], targets or []
        self.inputs_tensor_list, self.targets_tensor_list = [], []
        self.name = name or []

        # If a dataset_list is provided, merge the datasets
        if dataset_list:
            for d in dataset_list:
                self.inputs += d.inputs
                self.targets += d.targets
                self.inputs_tensor_list += d.inputs_tensor_list
                self.targets_tensor_list += d.targets_tensor_list

                if isinstance(d.name, list):
                    self.name.extend(d.name)
                else:
                    self.name.append(d.name)

            # Write merged data to a file
            processed_path = f"{d.args.data_dir}/{'+'.join(self.name)}_merged_processed.txt"
            with open(processed_path, "w", encoding='utf-8') as f:
                for i in range(len(self.inputs)):
                    f.write(f"{self.inputs[i]} ===> {self.targets[i]}\n")
            print(f"{self.name} is merged.")
            print(f"{processed_path} is written.")
        else:
            # If no dataset_list is provided, behave like ABSADataset
            self.max_len = args.max_seq_length
            self.inputs_tensor_list, self.targets_tensor_list = self.encode(self.inputs, self.targets)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs_tensor_list[index]["input_ids"].squeeze()
        target_ids = self.targets_tensor_list[index]["input_ids"].squeeze()

        src_mask = self.inputs_tensor_list[index]["attention_mask"].squeeze()
        target_mask = self.targets_tensor_list[index]["attention_mask"].squeeze()

        return {"source_ids": source_ids, "source_mask": src_mask,
                "target_ids": target_ids, "target_mask": target_mask}

    def encode(self, inputs=[], targets=[]):
        inputs_tensor_list, targets_tensor_list = [], []

        for i in range(len(inputs)):
            input_i = ' '.join(inputs[i]) if isinstance(inputs[i], list) else inputs[i]
            target_i = ' '.join(targets[i]) if isinstance(targets[i], list) else targets[i]


            # Tokenize input and target data
            tokenized_input = self.tokenizer.batch_encode_plus(
                [input_i], max_length=self.max_len, padding='max_length', truncation=True,
                return_tensors="pt",
            )
            tokenized_target = self.tokenizer.batch_encode_plus(
                [target_i], max_length=self.max_len, padding='max_length', truncation=True,
                return_tensors="pt"
            )

            inputs_tensor_list.append(tokenized_input)
            targets_tensor_list.append(tokenized_target)

        return inputs_tensor_list, targets_tensor_list


def get_inputs(args, data_type_file="train"):
    """
        train_inputs: ["hi", "I love apples."],
    """
    data_path = f"{args.data_dir}/{data_type_file}.txt"
    inputs, _ = read_line_examples_from_file(data_path)
#     inputs = [" ".join(i) for i in inputs]
    inputs = [ detokenize(" ".join(s)).strip() for s in inputs ]

    return inputs



def prepare_aste_extraction(args, data_type_file="train"):
    data_path = f"{args.data_dir}/{data_type_file}.txt"
    sents, labels = read_line_examples_from_file(data_path)
    inputs = [ detokenize(" ".join(s)).strip() for s in sents ]

    targets = []
    for i, label in enumerate(labels):
        all_tri = []
        for tri in label:
            if len(tri[0]) == 1:
                a = sents[i][tri[0][0]]
            else:
                start_idx, end_idx = tri[0][0], tri[0][-1]
                a = ' '.join(sents[i][start_idx:end_idx+1])
            if len(tri[1]) == 1:
                b = sents[i][tri[1][0]]
            else:
                start_idx, end_idx = tri[1][0], tri[1][-1]
                b = ' '.join(sents[i][start_idx:end_idx+1])
            c = senttag2word[tri[2]]
            a = detokenize(a).strip()
            b = detokenize(b).strip()
            all_tri.append((a, b, c))
        label_strs = ['( '+' , '.join(l)+' )' for l in all_tri]
        targets.append(' ; '.join(label_strs))
    return inputs, targets


def prepare_aste_universal(args, data_type_file="train"):
    data_path = f"{args.data_dir}/{data_type_file}.txt"
    sents, labels = read_line_examples_from_file(data_path)
    inputs = [ detokenize(" ".join(s)).strip() for s in sents ]

    targets = []
    for i, label in enumerate(labels):
        all_tri = []
        target_str = ""
        for tri in label:
            if len(tri[0]) == 1:
                aspect = sents[i][tri[0][0]]
            else:
                start_idx, end_idx = tri[0][0], tri[0][-1]
                aspect = ' '.join(sents[i][start_idx:end_idx+1])
            if len(tri[1]) == 1:
                opinion = sents[i][tri[1][0]]
            else:
                start_idx, end_idx = tri[1][0], tri[1][-1]
                opinion = ' '.join(sents[i][start_idx:end_idx+1])
            aspect = detokenize(aspect).strip()
            opinion = detokenize(opinion).strip()

            senti_token = TAG_TO_SPECIAL[tri[2]][0]
            target_str += f" {senti_token} {aspect} {OPINION_TOKEN} {opinion}"
        targets.append(target_str.strip())
    return inputs, targets

def detokenize(sentence: str) -> str:
    '''
    토큰화된 인풋을 원래 문장으로 복원
    '''
    ret = sentence.replace(" ", "")
    ret = ret.replace("▁", " ")

    return ret

def prepare_aste_gene(args, data_type_file):
    # just change order, since there is no none
    targets, inputs = prepare_aste_universal(args, data_type_file=data_type_file)
    return inputs, targets


def get_generation_inputs_and_targets(args, task, data_type):
    if data_type == "train":
        inputs, targets = prepare_aste_gene(args, data_type_file="train")
    else:
        raise NotImplementedError(f"Data type {data_type} is not supported.")

    return inputs, targets


def get_extraction_inputs_and_targets(args, task, data_type):
    task_mapping = {
        "aste": {"extraction": prepare_aste_extraction, "extraction-universal": prepare_aste_universal},
        "extract_aste": {"extraction": prepare_aste_extraction, "extraction-universal": prepare_aste_universal},
    }

    if task not in task_mapping:
        raise NotImplementedError

    if args.paradigm not in task_mapping[task]:
        raise NotImplementedError

    inputs, targets = task_mapping[task][args.paradigm](args, data_type_file=data_type)

    return inputs, targets


def get_inputs_and_targets(args, task, data_type):

    is_gene = True if "gene" in task else False
    if is_gene:
        inputs, targets = get_generation_inputs_and_targets(args, task, data_type)
    else:
        inputs, targets = get_extraction_inputs_and_targets(args, task, data_type)

    # rm label to prevent potential leakage
    if data_type == "target-unlabel":
        targets = ["Label removed." for t in targets]
        print("Removed label for target-unlabel.")

    path = f"{args.data_dir}/{data_type}_{task}_processed.txt"
    save_inputs_and_targets(path, inputs, targets)

    return inputs, targets


def save_inputs_and_targets(path, inputs, targets):
    with open(path, "w", encoding='utf-8') as f:
        for i in range(len(inputs)):
            f.write(f"{inputs[i]} ===> {targets[i]}\n")
    print(f"{path} is written.")


def get_dataset(args, task, data_type, tokenizer):
    inputs, targets = get_inputs_and_targets(args, task, data_type)
    dataset = ABSADataset(args, tokenizer, inputs=inputs, targets=targets, name=f"{data_type}_{task}")
    return dataset

In [ ]:
# eval_utils.py
import json
import re
import copy

import editdistance
import numpy as np

sentiment_word_list = ['positive', 'negative', 'neutral']

def extract_spans_extraction(task, seq, io_format):
    if task == "aste":
        if io_format == "extraction":
            return extract_aste_from_extraction(seq)
        elif io_format == "extraction-universal":
            return extract_aste_from_extraction_universal(seq)
        else:
            raise NotImplementedError


def extract_aste_from_extraction_universal(seq):
    aps = re.findall("(<pos>|<neg>|<neu>)(.+?)<opinion>(.+?)(?=<pos>|<neg>|<neu>|$)", seq)
    pairs = []
    for ap in aps:
        senti, aspect, opinion = ap[0].strip(), ap[1].strip(), ap[2].strip()
        senti = SPECIAL_TO_TAG[senti]
        pairs.append((aspect, opinion, senti))
    return pairs


def extract_aste_from_extraction(seq):
    aps = re.findall("\((.+?),(.+?),(.+?)\);?", seq)
    pairs = []
    for ap in aps:
        aspect, opinion, senti = ap[0].strip(), ap[1].strip(), ap[2].strip()
        pairs.append((aspect, opinion, senti))
    return pairs


def recover_terms_with_editdistance(original_term, sent):
    words = original_term.split(' ')
    new_words = []
    for word in words:
        edit_dis = []
        for token in sent:
            edit_dis.append(editdistance.eval(word, token))
        smallest_idx = edit_dis.index(min(edit_dis))
        new_words.append(sent[smallest_idx])
    new_term = detokenize(" ".join(new_words)).strip()
    return new_term


def fix_preds_aste(all_pairs, sents):

    all_new_pairs = []

    for i, pairs in enumerate(all_pairs):
        new_pairs = []
        if pairs == []:
            all_new_pairs.append(pairs)
        else:
            for pair in pairs:
                #two formats have different orders
                p0, p1, p2 = pair
                # for annotation-type
                if p1 in sentiment_word_list:
                    at, ott, ac = p0, p2, p1
                    io_format = 'annotation'
                # for extraction type
                elif p2 in sentiment_word_list:
                    at, ott, ac = p0, p1, p2
                    io_format = 'extraction'

                join_sent_i = detokenize(" ".join(sents[i])).strip()

                #print(pair)
                # AT not in the original sentence
                if at not in join_sent_i:
                    # print('Issue')
                    new_at = recover_terms_with_editdistance(at, sents[i])
                else:
                    new_at = at

                if ac not in sentiment_word_list:
                    new_sentiment = recover_terms_with_editdistance(ac, sentiment_word_list)
                else:
                    new_sentiment = ac

                # OT not in the original sentence
                ots = ott.split(', ')
                new_ot_list = []
                for ot in ots:
                    if ot not in join_sent_i:
                        # print('Issue')
                        new_ot_list.append(recover_terms_with_editdistance(ot, sents[i]))
                    else:
                        new_ot_list.append(ot)
                new_ot = ', '.join(new_ot_list)
                if io_format == 'extraction':
                    new_pairs.append((new_at, new_ot, new_sentiment))
                else:
                    new_pairs.append((new_at, new_sentiment, new_ot))
                # print(pair, '>>>>>', word_and_sentiment)
                # print(all_target_pairs[i])
            all_new_pairs.append(new_pairs)

    return all_new_pairs


def fix_pred_with_editdistance(all_predictions, sents, task):
    if task == "aste":
        fixed_preds = fix_preds_aste(all_predictions, sents)
    else:
        print("*** Unimplemented Error ***")
        fixed_preds = all_predictions

    return fixed_preds


def compute_f1_scores(pred_pt, gold_pt):
    """
    Function to compute F1 scores with pred and gold pairs/triplets
    The input needs to be already processed
    """
    # number of true postive, gold standard, predicted aspect terms
    n_tp, n_gold, n_pred = 0, 0, 0
    gold_pt = copy.deepcopy(gold_pt)

    for i in range(len(pred_pt)):
        n_gold += len(gold_pt[i])
        n_pred += len(pred_pt[i])

        for t in pred_pt[i]:
            if t in gold_pt[i]:
                # to prevent generate same correct answer and get recall larger than 1
                gold_pt[i].remove(t)
                n_tp += 1

    precision = float(n_tp) / float(n_pred) if n_pred != 0 else 0
    recall = float(n_tp) / float(n_gold) if n_gold != 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision != 0 or recall != 0 else 0
    scores = {'precision': precision, 'recall': recall, 'f1': f1}

    return scores


def compute_scores(pred_seqs, gold_seqs, sents, paradigm, task, verbose=False):
    """
    compute metrics for multiple tasks
    """
    assert len(pred_seqs) == len(gold_seqs)
    num_samples = len(gold_seqs)

    all_labels, all_predictions = [], []

    for i in range(num_samples):
        if "extraction" in paradigm:
            gold_list = extract_spans_extraction(task, gold_seqs[i], paradigm)
            pred_list = extract_spans_extraction(task, pred_seqs[i], paradigm)

            all_labels.append(gold_list)
            all_predictions.append(pred_list)

    raw_scores = compute_f1_scores(all_predictions, all_labels)
    # fix the issues due to generation
    all_predictions_fixed = fix_pred_with_editdistance(all_predictions, sents, task)
    fixed_scores = compute_f1_scores(all_predictions_fixed, all_labels)

    if verbose:
        for i in range(3):
            print(f"Gold: {gold_seqs[i]}")
            print(f"Gold list: {all_labels[i]}")
            print(f"Pred: {pred_seqs[i]}")
            print(f"Pred list: {all_predictions[i]}")
        print("Results of raw output")
        print(str(raw_scores))
        print("Results of fixed output")
        print(str(fixed_scores))

    return raw_scores, fixed_scores, all_labels, all_predictions, all_predictions_fixed


def avg_n_seeds_by_pair(output_dir, dirs, decode_txt, n_runs):
    score_avg_dict = {}
    score_type_list = ["raw_scores", "fixed_scores"]
    metric_list = ["precision", "recall", "f1"]
    pairs = []

    # collect value
    for dir_ in dirs:
        for score_type in score_type_list:
            if score_type not in score_avg_dict:
                score_avg_dict[score_type] = {}
            pair = dir_.split('/')[-1]
            src, tgt = pair.split('-')
            if pair not in score_avg_dict[score_type]:
                score_avg_dict[score_type][pair] = {}
            score_dict_i = json.load(open(f"{dir_}/score/test_{decode_txt}_score.json","r", encoding='UTF-8'))

            for metric in metric_list:
                if metric not in score_avg_dict[score_type][pair]:
                    score_avg_dict[score_type][pair][metric] = []
                score_avg_dict[score_type][pair][metric].append(score_dict_i[score_type][tgt][metric])

    # get all value
    for score_type in score_type_list:
        all_mat_dict = {k: [] for k in metric_list}
        for pair in score_avg_dict[score_type]:
            for metric in metric_list:
                f1_list_by_seed = score_avg_dict[score_type][pair][metric]
                all_mat_dict[metric].append(f1_list_by_seed)
        for metric in metric_list:
            if "all" not in score_avg_dict[score_type]:
                score_avg_dict[score_type]["all"] = {}
            score_avg_dict[score_type]["all"][metric] = np.mean(all_mat_dict[metric], axis=0)

    # avg value
    for score_type in score_type_list:
        for pair in score_avg_dict[score_type]:
            for metric in metric_list:
                mean = np.mean(score_avg_dict[score_type][pair][metric])
                std = np.std(score_avg_dict[score_type][pair][metric])
                score_avg_dict[score_type][pair][metric] = (mean, std)

    # visual result
    for score_type in score_type_list:
        print('@'*100)
        print(f"Avged {n_runs} runs {score_type}")
        print('\t'.join(list(score_avg_dict[score_type].keys())))
        f1_list = [i["f1"][0] for i in list(score_avg_dict[score_type].values())]
        print('\t'.join([f"{i*100:.2f}" for i in f1_list]))
        f1_std_list = [i["f1"][1] for i in list(score_avg_dict[score_type].values())]
        print('\t'.join([f"{i*100:.2f}" for i in f1_std_list]))

    json.dump(score_avg_dict, open(output_dir+f"/score_{decode_txt}_avg.json", "w", encoding='UTF-8'), indent=2, ensure_ascii=False)

In [ ]:
# preprocess.py
import os
import re

def write_generation(file_paths, split="train", data_dir=None, do_write=True, nrows=None, mode='aste'):

    final_sents = []
    count_dict = {}
    for path in file_paths:

        sentences, domain = None, None
        if mode == 'aste':
            sentences = open(path, "r", encoding='UTF-8').readlines()[:nrows]
            if split in ["dev", "test"]:
                domain = path.split('/')[-2]

        count_dict[domain] = len(sentences)
        final_sents.extend(sentences)
        print(f"{split} {path}: {len(sentences)}")

    print(f"{split} total: {len(final_sents)}")
    print(f"{split} count dict: {count_dict}")

    if do_write:
        output_dir = data_dir

        if not os.path.exists(output_dir):
            os.mkdir(output_dir)

        output_path = f'{output_dir}/{split}.txt'
        with open (output_path, 'w', encoding='utf-8') as f:
            for s in final_sents:
                f.write (f'{s.strip()}\n')
            print(f"{output_path} is written.")

    return final_sents, count_dict


def preprocess(dataset_dir=None, data_dir=None, source=None, targets=None, do_write=True, nrows=None, unlabel=False, mode='aste'):
    if mode == 'aste':
        train_file_format = "{i}/train.txt"
        dev_file_format = f"{source}/dev.txt"
        test_file_format = "{i}/test.txt"
    else:
        raise ValueError("Invalid mode. Choose 'aste'.")

    if unlabel:
        # Process unlabeled data
        train_paths = [os.path.join(dataset_dir, train_file_format.format(i=i)) for i in targets]
        train_sents, _ = write_generation(train_paths, "target-unlabel", data_dir=data_dir, do_write=do_write, nrows=nrows, mode=mode)
        print(f"Cross domain unlabel train_paths: {train_paths}")
        return train_sents

    else:
        # Process labeled data
        train_paths = [os.path.join(dataset_dir, train_file_format.format(i=i)) for i in [source]]
        dev_paths = [os.path.join(dataset_dir, dev_file_format)]
        test_paths = [os.path.join(dataset_dir, test_file_format.format(i=i)) for i in targets]

        print(f"train_paths: {train_paths}")
        print(f"dev_paths: {dev_paths}")
        print(f"test_paths: {test_paths}")

        # Read and preprocess the data using write_generation
        _, train_count_dict = write_generation(train_paths, "train", data_dir=data_dir, do_write=do_write, nrows=nrows, mode=mode)
        _, dev_count_dict = write_generation(dev_paths, "dev", data_dir=data_dir, do_write=do_write, nrows=nrows, mode=mode)
        _, test_count_dict = write_generation(test_paths, "test", data_dir=data_dir, do_write=do_write, nrows=nrows, mode=mode)

        return train_count_dict, dev_count_dict, test_count_dict


def prepare_raw_data(args):

    def process_task(preprocessor, mode, dataset_dir=None, data_gene=False, pseudo=False):
        train_count_dict, eval_count_dict, test_count_dict = preprocessor(
            dataset_dir=dataset_dir,
            data_dir=args.data_dir,
            source=args.source_domain,
            targets=args.target_domain,
            nrows=args.nrows,
            mode=mode
        )
        if data_gene or pseudo:
            preprocessor(
                dataset_dir=dataset_dir,
                data_dir=args.data_dir,
                source=args.source_domain,
                targets=args.target_domain,
                nrows=args.nrows,
                unlabel=True,
                mode=mode
            )
        return train_count_dict, eval_count_dict, test_count_dict

    task_map = {
        "aste": ('aste', "incre/aste/cross_domain")
    }

    if args.task in task_map and args.dataset == "cross_domain":
        mode, dataset_dir = task_map[args.task]
        return process_task(preprocess, mode, dataset_dir, args.data_gene, args.pseudo)
    else:
        raise NotImplementedError


In [ ]:
# model_utils.py
import torch

def prepare_constrained_tokens(tokenizer, task, paradigm):
    special_tokens = [tokenizer.eos_token] # add end token
    if task == "aste":
        if paradigm == "extraction-universal":
            special_tokens += [i[0] for i in TAG_TO_SPECIAL.values()]
            special_tokens += [OPINION_TOKEN]
        elif paradigm == "extraction":
            special_tokens += ["( a , a , negative ) ; ( a , a , neutral ) ; ( a , a , positive )"]
        else:
            raise NotImplementedError
    else:
        raise NotImplementedError
    return special_tokens


def prepare_tag_tokens(args):
    tag_tokens = []
    if args.task == "aste":
        if "extraction-universal" in args.paradigm:
            tag_tokens += [i[0] for i in TAG_TO_SPECIAL.values()]
            tag_tokens += [OPINION_TOKEN, SEP_TOKEN]
    else:
        raise NotImplementedError

    tag_tokens = list(set(tag_tokens))
    return tag_tokens


def init_tag(args, tokenizer, model, tag_tokens):
    if args.init_tag == "english":
        if args.paradigm == "extraction-universal":
            import re
            map_dict = {"pos": "positive", "neg": "negative", "neu": "neutral"}
            for tag_word in tag_tokens:
                tag_id = tokenizer.encode(tag_word, add_special_tokens=False)[0]
                init_word = re.sub("\W", "", tag_word).strip()
                # map senti
                if init_word in map_dict:
                    init_word = map_dict[init_word]
                # skip sep
                elif init_word == "sep":
                    continue
                init_id = tokenizer.encode(init_word, add_special_tokens=False)[0]
                with torch.no_grad():
                    model.shared.weight[tag_id] = model.shared.weight[init_id]
                print(f"{tokenizer.decode(tag_id)} is init by {tokenizer.decode(init_id)}")
        elif args.paradigm == "extraction":
            pass
        else:
            raise NotImplementedError
    else:
        raise NotImplementedError

In [ ]:
# run_utils.py
import re
import os
import random

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
# from tqdm import tqdm, trange
from tqdm.notebook import tqdm, trange
from transformers import (AutoModelForSeq2SeqLM,
                          T5Tokenizer, get_linear_schedule_with_warmup)

class Prefix_fn_cls():
    def __init__(self, tokenizer, special_tokens, input_enc_idxs):
        self.tokenizer=tokenizer
        self.input_enc_idxs=input_enc_idxs
        self.special_ids = [element for l in self.tokenizer(special_tokens, add_special_tokens=False)['input_ids'] for element in l]
        self.special_ids = list(set(self.special_ids))

    def get(self, batch_id, previous_tokens):
        # get input
        inputs = list(set(self.input_enc_idxs[batch_id].tolist()))+self.special_ids
        return inputs


def train(args, tokenizer, model, train_dataset, task, epochs, lr, bs, acc_step=None, save_ckpt=False, save_last=False):
    start_info = "#"*20+f" Conduct {task} Training"+"#"*20
    print("#"*len(start_info))
    print(start_info)
    print("#"*len(start_info))

    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        { "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay, },
        { "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0, },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=lr, eps=args.adam_epsilon)

    if acc_step is None:
        acc_step = args.gradient_accumulation_steps

    train_dataloader = DataLoader(train_dataset, batch_size=bs, drop_last=True, shuffle=True, num_workers=4)
    t_total = (
        (len(train_dataloader.dataset) // (bs * max(1, args.n_gpu)))
        // acc_step
        * float(epochs)
    )
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)
    train_iterator = trange(int(epochs), dynamic_ncols=True, desc="Epoch")

    # visualize input
    print(f"Training examples out of {len(train_dataset)}:")
    for i in range(3):
        print(f'Input : {tokenizer.decode(train_dataset[i]["source_ids"], skip_special_tokens=True)}')
        print(f'Output: {tokenizer.decode(train_dataset[i]["target_ids"], skip_special_tokens=True)}')

    print(f"Model emb weights of <pad> {model.shared.weight[0][:5]}")
    # start training
    for n_epoch, _ in enumerate(train_iterator):
        epoch_train_loss = 0.0
        for step, batch in enumerate(train_dataloader):
            model.train()

            lm_labels = batch["target_ids"]
            lm_labels[lm_labels[:, :] == tokenizer.pad_token_id] = -100


            outputs = model(
                batch["source_ids"].to(args.device),
                attention_mask=batch["source_mask"].to(args.device),
                labels=lm_labels.to(args.device),
                decoder_attention_mask=batch['target_mask'].to(args.device),
                decoder_input_ids=None,
            )

            loss = outputs[0]
            loss.backward()
            epoch_train_loss += loss.item()

            if (step+1) % acc_step == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                scheduler.step()
                model.zero_grad()

        if save_ckpt and n_epoch in range(args.num_train_epochs)[-args.save_last_k:]:
            ckpt_dir = os.path.join(args.seed_dir, f"checkpoint-e{n_epoch}")
            if not os.path.exists(ckpt_dir):
                os.makedirs(ckpt_dir)
            model.save_pretrained(ckpt_dir) # Output 용량 한계
            tokenizer.save_pretrained(ckpt_dir)
            # json.dump(args, open(f"{ckpt_dir}/args.json", 'w', encoding='UTF-8'), indent=4)
            print(f"Save model checkpoint to {ckpt_dir}")

        print(f"Epoch {n_epoch} Avg epoch train loss: {epoch_train_loss / len(train_dataloader):.5f} lr: {scheduler.get_last_lr()}")

    if save_last:
        ckpt_dir = os.path.join(args.seed_dir, f"{task}-model")
        if not os.path.exists(ckpt_dir):
            os.makedirs(ckpt_dir)
        model.save_pretrained(ckpt_dir)
        tokenizer.save_pretrained(ckpt_dir)
        print(f"Save model checkpoint to {ckpt_dir}")

    print("Finish training!")


def aux_training(args, tokenizer, model, train_dataset):

    return_values = {}
    if args.data_gene:
        return_values["data_gene_dataset"] = data_gene(args, tokenizer, model, train_dataset)
    if args.pseudo:
        return_values["pseudo_dataset"] = pseudo_label(args, tokenizer, model, train_dataset)

    return return_values


def infer(args, dataset, model, tokenizer, name, is_constrained=False, constrained_vocab=None, keep_mask=False, **decode_dict):
    dataloader = DataLoader(dataset, batch_size=args.eval_batch_size, num_workers=4)

    if keep_mask:
        # can't skip special directly, will lose extra_id
        unwanted_tokens = [tokenizer.eos_token, tokenizer.unk_token, tokenizer.pad_token]
        unwanted_ids = tokenizer.convert_tokens_to_ids(unwanted_tokens)
        def filter_decode(ids):
            ids = [i for i in ids if i not in unwanted_ids]
            tokens = tokenizer.convert_ids_to_tokens(ids)
            sentence = tokenizer.convert_tokens_to_string(tokens)
            return sentence

    # inference
    inputs, outputs, targets = [], [], []
    print(f"Inferencing on {name} ...")
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader):
            if is_constrained:
                prefix_fn_obj = Prefix_fn_cls(tokenizer, constrained_vocab, batch['source_ids'].to(args.device))
                prefix_fn = lambda batch_id, sent: prefix_fn_obj.get(batch_id, sent)
            else:
                prefix_fn = None

            outs_dict = model.generate(input_ids=batch['source_ids'].to(args.device),
                                        attention_mask=batch['source_mask'].to(args.device),
                                        max_length=64,
                                        prefix_allowed_tokens_fn=prefix_fn,
                                        output_scores=True,
                                        return_dict_in_generate=True,
                                        no_repeat_ngram_size=1,
                                        **decode_dict,
                                        )
            outs = outs_dict["sequences"]

            if keep_mask:
                input_ = [filter_decode(ids) for ids in batch["source_ids"]]
                dec = [filter_decode(ids) for ids in outs]
                target = [filter_decode(ids) for ids in batch["target_ids"]]
            else:
                input_ = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch["source_ids"]]
                dec = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
                target = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch["target_ids"]]

            inputs.extend(input_)
            outputs.extend(dec)
            targets.extend(target)

    decode_txt = "constrained" if is_constrained else "greedy"
    with open(os.path.join(args.inference_dir, f"{name}_{decode_txt}_output.txt"), "w", encoding='utf-8') as f:
        for i, o in enumerate(outputs):
            f.write(f"{inputs[i]} ===> {o}\n")

    return inputs, outputs, targets


def data_gene(args, tokenizer, model, train_dataset):

    # 1. extract label
    if args.data_gene_extract:
        extract_task = f"extract_{args.task}"
        target_extract_inputs, target_extract_outputs = extract_model(args, tokenizer, model, extract_task)

    # 2. gene train & infer
    target_gene_aug_inputs, target_gene_aug_outputs = gene_model(
        args, tokenizer, model, target_extract_inputs, target_extract_outputs
    )

    # 3. postprocess
    # change direction & post process
    target_gene_aug_inputs_processed, target_gene_aug_targets_processed = postprocess_gene_outputs(args, target_gene_aug_inputs, target_gene_aug_outputs)

    # 3.1 use the extract model to do filtering
    if args.model_filter:
        target_gene_aug_inputs_processed, target_gene_aug_targets_processed = model_filter(args, target_gene_aug_inputs_processed, target_gene_aug_targets_processed)
        print(f"Aug num after filtering: {len(target_gene_aug_inputs_processed)}")

    # 3.2 control number of augmentation number
    if args.data_gene_aug_num:
        target_gene_aug_inputs_processed, target_gene_aug_targets_processed = \
            target_gene_aug_inputs_processed[:args.data_gene_aug_num], target_gene_aug_targets_processed[:args.data_gene_aug_num]
    if args.data_gene_aug_ratio:
        aug_num = int(len(target_gene_aug_inputs_processed) * args.data_gene_aug_ratio)
        target_gene_aug_inputs_processed, target_gene_aug_targets_processed = \
            target_gene_aug_inputs_processed[:aug_num], target_gene_aug_targets_processed[:aug_num]

    print(f"Aug num final: {len(target_gene_aug_inputs_processed)}")

    # 4. merge dataset
    target_gene_aug_dataset = ABSADataset(args, tokenizer, inputs=target_gene_aug_inputs_processed, targets=target_gene_aug_targets_processed, name="target_gene_aug")
    train_dataset_merged = ABSADataset(args, tokenizer, dataset_list=[train_dataset, target_gene_aug_dataset])

    return train_dataset_merged


def pseudo_label(args, tokenizer, model, train_dataset):
    # 1. train absa on train
    train(args, tokenizer, model, train_dataset, task=f"pseudo_{args.task}", epochs=args.num_train_epochs, lr=args.learning_rate, bs=args.train_batch_size, save_ckpt=False, save_last=True)

    # 2. inference on target unlabel
    target_dataset = get_dataset(args, task=args.task, data_type="target-unlabel", tokenizer=tokenizer)
    target_pseudo_inputs, target_pseudo_outputs, _ = infer(
        args, target_dataset, model, tokenizer, name=f"target_pseudo_{args.task}",
        is_constrained=True, constrained_vocab=prepare_constrained_tokens(tokenizer, args.task, args.paradigm),
    )

    if args.pseudo_skip_none:
        target_pseudo_inputs, target_pseudo_outputs = pseudo_filter_none(target_pseudo_inputs, target_pseudo_outputs)

    # 3. merge pseudo labelled data
    target_pseudo_aug_dataset = ABSADataset(args, tokenizer, inputs=target_pseudo_inputs, targets=target_pseudo_outputs, name="target_pseudo_absa")
    train_dataset_merged = ABSADataset(args, tokenizer, dataset_list=[train_dataset, target_pseudo_aug_dataset])
    return train_dataset_merged


def pseudo_filter_none(inputs, outputs):
    new_inputs, new_outputs = [], []
    for idx in range(len(outputs)):
        if "none" not in outputs[idx]:
            new_inputs.append(inputs[idx])
            new_outputs.append(outputs[idx])
    return new_inputs, new_outputs


def extract_model(args, tokenizer, model, extract_task):

    # 1. train extract model
    if args.extract_model:
        model = AutoModelForSeq2SeqLM.from_pretrained(args.extract_model).to(args.device)
        tokenizer = T5Tokenizer.from_pretrained(args.extract_model, legacy=False)
        print(f"Model reloaded with {args.extract_model}")
        print(f"Tokenizer len: {len(tokenizer)}")
    elif args.runned_folder:
        model_path = os.path.join(args.runned_folder, f"seed-{args.seed}",
        f"{args.source_domain}-{args.target_domain[0]}", f"extract_{args.task}-model")
        model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(args.device)
        tokenizer = T5Tokenizer.from_pretrained(model_path, legacy=False)
        print(f"Model reloaded with {model_path}")

    train_extract_dataset = get_dataset(args, task=extract_task, data_type="train", tokenizer=tokenizer)
    train(args, tokenizer, model, train_extract_dataset, task=extract_task, epochs=args.data_gene_extract_epochs, lr=args.learning_rate, bs=args.train_batch_size, save_ckpt=False, save_last=True)

    # 2. infer on target domain
    target_extract_dataset = get_dataset(args, task=extract_task, data_type="target-unlabel", tokenizer=tokenizer)
    target_extract_inputs, target_extract_outputs, _ = infer(
        args, target_extract_dataset, model, tokenizer,
        name=f"target_{extract_task}", is_constrained=True, constrained_vocab=prepare_tag_tokens(args)
    )
    return target_extract_inputs, target_extract_outputs


def gene_model(args, tokenizer, model, target_extract_inputs, target_extract_outputs):

    if args.gene_model:
        model = AutoModelForSeq2SeqLM.from_pretrained(args.gene_model).to(args.device)
        tokenizer = T5Tokenizer.from_pretrained(args.gene_model, legacy=False)
        print(f"Model reloaded with {args.gene_model}")
    elif args.runned_folder:
        model_path = os.path.join(args.runned_folder, f"seed-{args.seed}",
        f"{args.source_domain}-{args.target_domain[0]}", f"gene_{args.task}-model")
        model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(args.device)
        tokenizer = T5Tokenizer.from_pretrained(model_path, legacy=False)
        print(f"Model reloaded with {model_path}")
    # 0. load a new model
    elif args.data_gene_same_model or args.use_same_model:
        print(f"Model keep the same.")
    else:
        model = AutoModelForSeq2SeqLM.from_pretrained(args.model_name_or_path).to(args.device)
        model.resize_token_embeddings(len(tokenizer))
        print(f"Model reloaded with {args.model_name_or_path}")
    print(f"Tokenizer len: {len(tokenizer)}")

    # 1. train gene model
    train_gene_dataset = get_dataset(args, task=f"gene_{args.task}", data_type="train", tokenizer=tokenizer)
    train(args, tokenizer, model, train_gene_dataset, task=f"gene_{args.task}", epochs=args.data_gene_epochs, lr=args.learning_rate, bs=args.train_batch_size, save_ckpt=False, save_last=True)

    # 2. infer gene model
    # 2.0 prepare infer dataset
    if args.data_gene_extract:
        target_gene_inputs, target_gene_targets = target_extract_outputs, target_extract_inputs
        target_gene_dataset = ABSADataset(args, tokenizer, inputs=target_gene_inputs, targets=target_gene_targets, name="target_gene")

    # 2.1 constrained decoding, but may not be used depends on args.data_gene_wt_constrained
    target_domain_words = prepare_gene_vocab(args, tokenizer)

    # 2.2 inference
    decode_dict = {"min_length": args.data_gene_min_length,}
    specific_dict = {
        "greedy": {"do_sample": False},
        "top_p": {"do_sample": True, "top_p": args.data_gene_top_p},
        "beam": {"num_beams": args.data_gene_num_beam, "early_stopping": True},
    }
    if args.data_gene_decode:
        decode_dict.update(specific_dict[args.data_gene_decode])

    is_constrained = False if args.data_gene_wt_constrained else True
    target_gene_aug_inputs, target_gene_aug_outputs, _ = infer(
        args, target_gene_dataset, model, tokenizer, name="target_gene",
        is_constrained=is_constrained, constrained_vocab=target_domain_words, **decode_dict
    )

    return target_gene_aug_inputs, target_gene_aug_outputs


def postprocess_gene_outputs(args, target_gene_aug_inputs, target_gene_aug_outputs):
    # process input & output
    target_gene_aug_inputs_processed = target_gene_aug_outputs
    target_gene_aug_targets_processed = [i.strip() for i in target_gene_aug_inputs]

    # normalize and filter
    target_gene_aug_inputs_processed, target_gene_aug_targets_processed = normalize_augment(args, target_gene_aug_inputs_processed, target_gene_aug_targets_processed)
    target_gene_aug_inputs_processed, target_gene_aug_targets_processed = filter_none(target_gene_aug_inputs_processed, target_gene_aug_targets_processed, args.data_gene_none_remove_ratio)
    target_gene_aug_inputs_processed, target_gene_aug_targets_processed = filter_invalid(target_gene_aug_inputs_processed, target_gene_aug_targets_processed)

    return target_gene_aug_inputs_processed, target_gene_aug_targets_processed


def extract_label_words(inputs):
    label_words = []
    for i in inputs:
        i = re.sub("<\w+>", "", i)
        label_words += i.strip().split()
    return label_words


def prepare_gene_vocab(args, tokenizer):
    # Get target inputs
    target_inputs = get_inputs(args, data_type_file="target-unlabel")
    # Create a set to avoid duplicates and improve performance
    target_tokens = []
    for target_i in target_inputs:
        tokens = tokenizer.tokenize(target_i)
        target_tokens.extend(tokens)
    target_domain_words = set(target_tokens)
    # Extend with tag tokens
    target_domain_words.update(prepare_tag_tokens(args))
    print(f"{len(target_domain_words)} target domain words")
    target_domain_words = list(target_domain_words)
    target_domain_words.extend(STOP_WORDS)

    return target_domain_words


def model_filter(args, inputs, outputs):

    extract_path = os.path.join(args.seed_dir, f"extract_{args.task}-model")
    model2 = AutoModelForSeq2SeqLM.from_pretrained(extract_path).to(args.device)
    tokenizer2 = T5Tokenizer.from_pretrained(extract_path, legacy=False)
    print(f"{extract_path} loaded.")
    print(f"Model emb weights of <pad> {model2.shared.weight[0][:5]}")

    filter_dataset = ABSADataset(args, tokenizer2, inputs=inputs, targets=outputs, name="target_filter")
    filter_inputs, filter_outputs, _  = infer(
        args, filter_dataset, model2, tokenizer2,
        name=f"target_filter", is_constrained=True, constrained_vocab=prepare_tag_tokens(args)
    )

    assert len(filter_inputs) == len(inputs)

    removed = []

    new_inputs, new_outputs = [], []
    filter_num = 0
    for i in range(len(outputs)):
        if i < 5:
            print(f'filter:{filter_outputs[i].strip()}, output:{outputs[i].strip()}')
        if filter_outputs[i].strip() != outputs[i].strip():
            filter_num += 1
            removed.append(' #### '.join([inputs[i], outputs[i], filter_outputs[i]]))
            continue
        else:
            new_inputs.append(inputs[i])
            new_outputs.append(outputs[i])

    print(f"{filter_num} augmentations out of {len(inputs)} are removed by model.")

    with open(os.path.join(args.inference_dir, f"model_filter.txt"), "w", encoding='utf-8') as f:
        for i, o in enumerate(removed):
            f.write(f"{o}\n")

    return new_inputs, new_outputs

In [ ]:
# setup.py
import argparse
import os
import random
from datetime import datetime

def init_args():
    parser = argparse.ArgumentParser()
    # basic settings
    parser.add_argument("--task", default='uabsa', type=str, required=True,
                        help="The name of the task, selected from: [xabsa]")
    parser.add_argument("--device", default='cuda', type=str, required=False)
    parser.add_argument("--dataset", default='cross_domain', type=str, required=True,
                        help="The name of the dataset, selected from: [cross_domain]")
    parser.add_argument("--data_dir", default=None, type=str, help="args.output_dir/data")
    parser.add_argument("--model_name_or_path", default='mt5-base', type=str,
                        help="Path to pre-trained model or shortcut name")
    parser.add_argument("--paradigm", default='annotation', type=str, required=True,
                        help="""The way to construct target sentence, selected from:[
                            extraction:             (apple, positive); (banana, negative) //  None
                            extraction-universal:  <pos> apple <pos> orange <neg> banana //   [none]
                            ]""")
    parser.add_argument("--do_train", action='store_true', help="Whether to run training.")
    parser.add_argument("--do_eval", action='store_true', help="Whether to run eval on the dev/test set.")
    parser.add_argument("--name", default='experinments', type=str, help="name of the exp")
    parser.add_argument("--commit", default=None, type=str, help="commit id")
    parser.add_argument("--save_last_k", default=5, type=int, help="save last k")
    parser.add_argument("--n_runs", default=5, type=int, help="run with n seeds")
    parser.add_argument("--clear_model", action='store_true', help="remove saved ckpts")
    parser.add_argument("--save_best", action='store_true', help="save best model only")
    parser.add_argument("--nrows", default=None, type=int, help="# of lines to be read")

    # basic setting
    parser.add_argument("--train_by_pair", action='store_true', help="train a model for each pair")
    parser.add_argument("--target_domain", default=None, type=str)

    # inference
    parser.add_argument("--no_greedy", action='store_true', help="only constrained decoding")
    parser.add_argument("--beam", default=1, type=int)

    # Other parameters
    parser.add_argument("--max_seq_length", default=128, type=int)
    parser.add_argument("--n_gpu", default=0)
    parser.add_argument("--train_batch_size", default=16, type=int,
                        help="Batch size per GPU/CPU for training.")
    parser.add_argument("--eval_batch_size", default=16, type=int,
                        help="Batch size per GPU/CPU for evaluation.")
    parser.add_argument('--gradient_accumulation_steps', type=int, default=1,
                        help="Number of updates steps to accumulate before performing a backward/update pass.")
    parser.add_argument("--learning_rate", default=3e-4, type=float)
    parser.add_argument("--num_train_epochs", default=20, type=int,
                        help="Total number of training epochs to perform.")
    parser.add_argument('--seed', type=int, default=42, help="random seed for initialization")

    # training details
    parser.add_argument("--weight_decay", default=0.0, type=float)
    parser.add_argument("--adam_epsilon", default=1e-8, type=float)
    parser.add_argument("--warmup_steps", default=0.0, type=float)

    # template
    parser.add_argument("--init_tag", default=None, type=str, help="[english]")

    # data genenration
    parser.add_argument("--data_gene", action='store_true', help="enable data generation")
    parser.add_argument("--data_gene_epochs", default=0, type=int, help="epoch of generation model training")
    parser.add_argument("--data_gene_extract", action='store_true', help="enable text-to-label stage")
    parser.add_argument("--data_gene_extract_epochs", default=0, type=int, help="epoch of extract model training")
    parser.add_argument("--data_gene_none_remove_ratio", default=0, type=float, help="remove none input for gene model")
    parser.add_argument("--data_gene_none_word_num", default=1, type=int, help="rand word added to generate diverse none examples")
    parser.add_argument("--data_gene_extract_none_remove_ratio", default=0, type=float, help="remove none training sample for extract model training")
    parser.add_argument("--data_gene_same_model", action='store_true', help="extract & gene model share the same model")
    parser.add_argument("--data_gene_wt_constrained", action='store_true', help="turn off constrained decoding during gene generation")
    parser.add_argument("--use_same_model", action='store_true', help="all stages use the same model")
    parser.add_argument("--model_filter", action='store_true', help="use extract model inference for filtering")
    parser.add_argument("--model_filter_skip_none", action='store_true', help="keep the sample if extract model output none")

    # decode
    parser.add_argument("--data_gene_decode", default=None, type=str, help="[greedy, top_p, beam]")
    parser.add_argument("--data_gene_top_p", default=0.9, type=float)
    parser.add_argument("--data_gene_num_beam", default=1, type=int)
    parser.add_argument("--data_gene_min_length", default=0, type=int)
    parser.add_argument("--extract_model", default=None, type=str, help="path to extract model")
    parser.add_argument("--gene_model", default=None, type=str, help="path to gene model")

    parser.add_argument("--runned_folder", default=None, type=str, help="Load previous trained model for aux training")
    parser.add_argument("--data_gene_aug_num", default=None, type=int, help="how many augmentation samples")
    parser.add_argument("--data_gene_aug_ratio", default=None, type=float, help="how much ratio of augmentation samples")

    # pseudo labelling
    parser.add_argument("--pseudo", action='store_true', help="data generation")
    parser.add_argument("--pseudo_skip_none", action='store_true', help="use non-none only")

    args = parser.parse_args()

    # Set up output directory
    output_dir = f"../outputs/{args.task}/{args.dataset}"
    os.makedirs(output_dir, exist_ok=True)

    # Create a timestamped directory
    timestamp = datetime.now().strftime("%m%d_%H%M")
    output_dir = os.path.join(output_dir, f"{timestamp}-{args.name}")
    os.makedirs(output_dir, exist_ok=True)

    args.output_dir = output_dir

    return args


def prepare_seeds(seed=42, n_runs=1):
    # prepare n seeds
    random.seed(seed)
    seed_list = random.sample(range(100), n_runs-1)
    seed_list += [seed]
    seed_list = sorted(seed_list)
    print(f"Seed list: {seed_list}")
    return seed_list


def prepare_pairs(args):
    # Define the mapping between tasks, datasets, and pairs
    task_dataset_pairs = {
        "aste": {"cross_domain": ASTE_TRANSFER_PAIRS}
    }

    # Check if the task and dataset combination is valid and retrieve the corresponding pair
    pair_dict = task_dataset_pairs.get(args.task, {}).get(args.dataset)

    if pair_dict is None:
        raise NotImplementedError("The task and/or dataset is not implemented.")

    return pair_dict


In [ ]:
# main.py
import json
import os
import sys

from pytorch_lightning import seed_everything
from transformers import (AutoModelForSeq2SeqLM, T5Tokenizer)

def prepare_logger(args):
    # Setup logging
    logging.basicConfig(format='%(asctime)s - %(name)s - %(message)s', datefmt='%m/%d/%Y %H:%M:%S', level=logging.INFO)
    formatter = logFormatter = logging.Formatter(fmt='[%(asctime)s - %(name)s:%(lineno)d]: %(message)s', datefmt='%m/%d/%Y %H:%M:%S')
    log_file = os.path.join(args.seed_dir, "run.log")
    file_handler = logging.FileHandler(log_file, mode="w", encoding='utf-8', delay=False)
    console_handler = logging.StreamHandler(sys.stdout)
    file_handler.setFormatter(formatter)
    console_handler.setFormatter(formatter)
    logger.handlers = [console_handler, file_handler]


def evaluate(args, tokenizer, dataset, model, paradigm, task, sents, split, is_constrained, eval_set_count_dict=None, silent=False):
    """
    Compute scores given the predictions and gold labels
    """

    decode_txt = "constrained" if is_constrained else "greedy"
    print(f"Eval set count dict: {eval_set_count_dict}")

    inputs, outputs, targets = infer(
        args, dataset, model, tokenizer, name=f"{split}_{task}",
        is_constrained=is_constrained, constrained_vocab=prepare_constrained_tokens(tokenizer, task, paradigm),
    )

    start_idx = 0
    score_dict = {
        "raw_scores": {k: 0 for k in eval_set_count_dict.keys()},
        "fixed_scores": {k: 0 for k in eval_set_count_dict.keys()},
    }
    pred_dict = {
        "labels": {k: 0 for k in eval_set_count_dict.keys()},
        "preds": {k: 0 for k in eval_set_count_dict.keys()},
        "preds_fixed": {k: 0 for k in eval_set_count_dict.keys()},
    }

    verbose = True if split == "dev" else False
    for l, num in eval_set_count_dict.items():
        raw_score, fixed_score, label, pred, pred_fixed = compute_scores(
                                                outputs[start_idx: start_idx+num],
                                                targets[start_idx: start_idx+num],
                                                sents[start_idx: start_idx+num],
                                                paradigm, task, verbose)
        start_idx += num

        score_dict["raw_scores"][l] = raw_score
        score_dict["fixed_scores"][l] = fixed_score
        pred_dict["labels"][l] = label
        pred_dict["preds"][l] = pred
        pred_dict["preds_fixed"][l] = pred_fixed

    """
        score_dict = {
            "raw_scores": {
                "en": {"precision": 0, "recall": 0, "f1": 0},
                "all": {"precision": 0, "recall": 0, "f1": 0},
            }
            "fixed_score": {
            }
        }
    """

    if not silent:
        for score_type in ["raw_scores", "fixed_scores"]:

            print('='*100)
            print(score_type)
            print('\t'.join(list(score_dict[score_type].keys())))
            f1_list = [i["f1"] for i in list(score_dict[score_type].values())]
            print('\t'.join([f"{i*100:.2f}" for i in f1_list]))

    with open(os.path.join(args.score_dir, f"{split}_{decode_txt}_errors.txt"), "w", encoding='utf-8') as f:
        counter = 0
        for lang in pred_dict['labels'].keys():
            for i in range(len(pred_dict['labels'][lang])):
                label_i = pred_dict['labels'][lang][i]
                pred_i = pred_dict['preds'][lang][i]
                pred_fixed_i = pred_dict['preds_fixed'][lang][i]
                test_i = ' '.join(sents[counter])
                if label_i != pred_i:
                    f.write(f"{test_i} === {label_i} === {pred_i} === {pred_fixed_i} \n")
                counter += 1

    # return  {k: score_dict["fixed_scores"]["all w/t en"][k] for k in ["precision", "recall", "f1"]}
    return score_dict, pred_dict


def main(args):
    prepare_logger(args)
    start_info = "="*30+f"NEW EXP: {args.task.upper()} on {args.dataset} with seed {args.seed}"+"="*30
    print("#"*len(start_info))
    print(start_info)
    print("#"*len(start_info))

    # Initialize args
    args.ori_tok_len = T5_ORI_LEN
    args.data_dir = os.path.join(args.seed_dir, "data")
    args.inference_dir = os.path.join(args.seed_dir, "inference")
    args.score_dir = os.path.join(args.seed_dir, "score")

    # Create inference and score directories if they do not exist
    for directory in [args.inference_dir, args.score_dir]:
        os.makedirs(directory, exist_ok=True)

    # Log and save args
    print(args)
    json.dump(args.__dict__, open(args.seed_dir+"/args.json", 'w', encoding='UTF-8'), indent=2, ensure_ascii=False)

    seed_everything(args.seed)

    # prepare raw data, inlcudes [train.txt, test.txt, dev.txt, target-unlabel.txt] etc.
    train_count_dict, eval_count_dict, test_count_dict = prepare_raw_data(args)

    tag_tokens = prepare_tag_tokens(args)

    tokenizer = T5Tokenizer.from_pretrained(args.model_name_or_path, legacy=False)
    model = AutoModelForSeq2SeqLM.from_pretrained(args.model_name_or_path)

    # training process
    if args.do_train:
        # add special tokens
        special_tokens = tag_tokens
        tokenizer.add_tokens(special_tokens)
        model.resize_token_embeddings(len(tokenizer))
        print(f"Tokens added into embedding: {special_tokens}")
        print(f"Tokenizer len: {len(tokenizer)}")

        if args.init_tag:
            init_tag(args, tokenizer, model, tag_tokens)

        # show one sample to check the sanity of the code and the expected output
        print(f"Here is an example (from dev set) under `{args.paradigm}` paradigm:")
        dev_dataset = get_dataset(args, task=args.task, data_type="dev", tokenizer=tokenizer)
        for i in range(3):
            print(f'Input : {tokenizer.decode(dev_dataset[i]["source_ids"], skip_special_tokens=True)}')
            print(f'Output: {tokenizer.decode(dev_dataset[i]["target_ids"], skip_special_tokens=True)}')

        model.to(args.device)

        train_dataset = get_dataset(args, task=args.task, data_type="train", tokenizer=tokenizer)

        # aux training, inlcude [data_gene, pseudo]
        aux_outputs = aux_training(args, tokenizer, model, train_dataset)
        if args.data_gene or args.pseudo:
            if args.data_gene:
                train_dataset = aux_outputs["data_gene_dataset"]
            if args.pseudo:
                train_dataset = aux_outputs["pseudo_dataset"]
            # reload model
            if args.use_same_model:
                print(f"Use the same model.")
            else:
                model = AutoModelForSeq2SeqLM.from_pretrained(args.model_name_or_path)
                model.to(args.device)
                print(f"Model reloaded with {args.model_name_or_path}")
                model.resize_token_embeddings(len(tokenizer))
                print(f"Tokens added into embedding: {special_tokens}")

        # official training
        train(args, tokenizer, model, train_dataset, task=args.task, epochs=args.num_train_epochs, lr=args.learning_rate,
        bs=args.train_batch_size, save_ckpt=True)

    if args.do_eval:

        print("*"*20+" Conduct Evaluating"+"*"*20)

        all_checkpoints = []
        # retrieve all the saved checkpoints for model selection
        saved_model_dir = args.seed_dir
        if args.do_train:
            for f in os.listdir(saved_model_dir):
                file_name = os.path.join(saved_model_dir, f)
                if 'checkpoint' in file_name:
                    all_checkpoints.append(file_name)
        else:
            all_checkpoints.append(args.model_name_or_path)
            tokenizer = T5Tokenizer.from_pretrained(args.model_name_or_path, legacy=False)

        # conduct some selection (or not)
        print(f"We will perform validation on the following checkpoints: {all_checkpoints}")

        # load dev and test datasets
        test_sents, _ = read_line_examples_from_file(f"{args.data_dir}/test.txt")
        dev_sents, _ = read_line_examples_from_file(f"{args.data_dir}/dev.txt")
        train_sents, _ = read_line_examples_from_file(f"{args.data_dir}/train.txt")

        dev_dataset = get_dataset(args, task=args.task, data_type="dev", tokenizer=tokenizer)
        test_dataset = get_dataset(args, task=args.task, data_type="test", tokenizer=tokenizer)
        train_dataset = get_dataset(
            args, task=args.task, data_type="train", tokenizer=tokenizer)

        decode_list = [False, True]
        if args.no_greedy:
            decode_list = [True]
        for is_constrained in decode_list:

            decode_txt = "constrained" if is_constrained else "greedy"
            print("%"*100)
            print(f"Decode by {decode_txt}")
            best_f1, best_checkpoint, best_epoch = -999999.0, None, None
            best_score_dict, best_pred_dict = None, None
            all_epochs = []

            score_dicts = { "dev": [], "test": []}

            for checkpoint in all_checkpoints:
                epoch = checkpoint.split('-')[-1][1:]
                all_epochs.append(epoch)

                # reload the model and conduct inference
                print(f"Load the trained model from {checkpoint}...")
                model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
                model.to(args.device)

                score_dict_dev, pred_dict_dev = evaluate(args, tokenizer, dev_dataset, model, args.paradigm, args.task, dev_sents, "dev",
                                                        is_constrained=is_constrained, eval_set_count_dict=eval_count_dict, silent=False)
                score_dict_test, pred_dict_test = evaluate(args, tokenizer, test_dataset, model, args.paradigm, args.task, test_sents, "test",
                                                        is_constrained=is_constrained, eval_set_count_dict=test_count_dict, silent=False)
                score_dict_train, pred_dict_train = evaluate(
                    args, tokenizer, train_dataset,
                    model, args.paradigm, args.task,
                    train_sents, "train",
                    is_constrained=is_constrained,
                    eval_set_count_dict=train_count_dict,
                    silent=True
                )
                if args.dataset == "cross_domain":
                    best_condition = score_dict_dev["raw_scores"][args.source_domain]['f1']
                if best_condition > best_f1:
                    best_f1 = best_condition
                    best_checkpoint = checkpoint
                    best_epoch = epoch
                    best_score_dict = score_dict_test
                    best_pred_dict = pred_dict_test

                score_dicts["dev"].append(score_dict_dev)
                score_dicts["test"].append(score_dict_test)

            json.dump(best_score_dict, open(f"{args.score_dir}/test_{decode_txt}_score.json", 'w', encoding='UTF-8'), indent=4, ensure_ascii=False)
            json.dump(best_pred_dict, open(f"{args.score_dir}/test_{decode_txt}_pred.json", 'w', encoding='UTF-8'), indent=4, ensure_ascii=False)

            # visualize evaluation results
            for split_i in ["dev", "test"]:
                separator = "\t"
                print('='*100)
                print(f"{split_i} result over from epochs {all_epochs}")
                for score_type in ["raw_scores", "fixed_scores"]:
                    print(score_type)
                    langs = list(score_dicts[split_i][0][score_type].keys())
                    print(f"lang{separator}"+f"{separator}".join(langs))
                    for i, epoch_i in enumerate(all_epochs):
                        if epoch_i == best_epoch:
                            print_line = f"epoch-{epoch_i}[best]{separator}"
                        else:
                            print_line = f"epoch-{epoch_i}{separator}"
                        for lang in langs:
                            for metric_i in ["precision", "recall", "f1"]:
                                print_line += "{:.2f}".format(100*score_dicts[split_i][i][score_type][lang][metric_i])+"/"
                            print_line += separator
                        print(print_line)

            # print test results over last few steps
            print(f"The best checkpoint is {best_checkpoint}")

        # only training's model needs to be deleted
        if args.clear_model and args.do_train:
            import shutil
            for checkpoint in all_checkpoints:
                if args.save_best and checkpoint == best_checkpoint:
                        continue
                else:
                    shutil.rmtree(checkpoint)
                    print(f"{checkpoint} is removed")


def collate_seed_results(args, runed_dirs):
    decode_txt_list = ["constrained"] if args.no_greedy else ["greedy", "constrained"]
    for decode_txt in decode_txt_list:
        print(f"Averaging {decode_txt}")
        if args.train_by_pair:
            avg_n_seeds_by_pair(args.output_dir, runed_dirs, decode_txt, args.n_runs)
        else:
            raise NotImplementedError


def run_multiple_seeds(args, seed_list):

    runed_dirs = []

    for i in range(args.n_runs):

        print(f"Running with seed {seed_list[i]}")

        pair_dict = prepare_pairs(args)
        for source in pair_dict:
            if args.train_by_pair:
                for target in pair_dict[source]:
                    start_info = "#"*20+f"Working on {source} --> {target}"+"$"*20
                    print("#"*len(start_info))
                    print(start_info)
                    print("#"*len(start_info))

                    output_dir_i = f"{args.output_dir}/seed-{seed_list[i]}/{source}-{target}"
                    os.makedirs(output_dir_i, exist_ok=True)
                    runed_dirs.append(output_dir_i)
                    args.seed_dir, args.seed = output_dir_i, seed_list[i]
                    args.source_domain, args.target_domain = source, [target]
                    main(args)
            else:
                raise NotImplementedError

    return runed_dirs


In [ ]:
from datetime import datetime

class ABSAConfig():
    task = 'aste'
    device = 'cuda'
    dataset = 'cross_domain'
    data_dir = 'incre'
    model_name_or_path = 'KETI-AIR/ke-t5-base-ko'  # 'mt5-base' # mt5-base가 다국어
    paradigm = 'extraction-universal'  # 'annotation'
    do_train = True  # action='store_true'
    do_eval = True  # action='store_true'
    name = 'self-check'
    commit = None
    save_last_k = 4  # 5
    n_runs = 1  # 5
    clear_model = True  # action='store_true'
    save_best = True  # action='store_true'
    nrows = None

    # basic setting
    train_by_pair = True  # action='store_true'
    target_domain = None

    # inference
    no_greedy = True # True로 설정하면 is_constrained 만 # action='store_true'
    beam = 1

    # Other parameters
    max_seq_length = 64
    n_gpu = 0
    train_batch_size = 4  # 16
    eval_batch_size = 4
    gradient_accumulation_steps = 2  # 1
    learning_rate = 5e-4
    num_train_epochs = 20  # 20
    seed = 1398

    # training details
    weight_decay = 0
    adam_epsilon = 1e-8
    warmup_steps = 0

    # template
    init_tag = 'english'  # None

    # data genenration
    data_gene = True  # action='store_true'
    data_gene_epochs = 25 # 25  # 0
    data_gene_extract = True  # action='store_true'
    data_gene_extract_epochs = 25 # 25  # 0
    data_gene_none_remove_ratio = 0
    data_gene_none_word_num = 1
    data_gene_extract_none_remove_ratio = 0
    data_gene_same_model = False  # action='store_true'
    data_gene_wt_constrained = True  # action='store_true'
    use_same_model = True  # action='store_true'
    model_filter = True  # action='store_true'
    model_filter_skip_none = False  # action='store_true'

    # decode
    data_gene_decode = 'top_p' # [greedy, top_p, beam]
    data_gene_top_p = 0.92
    data_gene_num_beam = 2
    data_gene_min_length = 0
    extract_model = None
    gene_model = None

    runned_folder = None
    data_gene_aug_num = None
    data_gene_aug_ratio = None

    # pseudo labelling
    pseudo = False  # action='store_true'
    pseudo_skip_none = False  # action='store_true'
    def __init__(self):
      # Set up output directory
      output_dir = f"{self.task}/{self.dataset}"
      os.makedirs(output_dir, exist_ok=True)

      # Create a timestamped directory
      timestamp = datetime.now().strftime("%m%d_%H%M")
      output_dir = os.path.join(output_dir, f"{timestamp}-{self.name}")
      os.makedirs(output_dir, exist_ok=True)

      self.output_dir = output_dir

In [ ]:
absa_config = ABSAConfig()

In [ ]:
#args = init_args()
seed_list = prepare_seeds(absa_config.seed, absa_config.n_runs)
runed_dirs = run_multiple_seeds(absa_config, seed_list)

Seed list: [1398]
Running with seed 1398
##################################################################
####################Working on total --> total$$$$$$$$$$$$$$$$$$$$
##################################################################
########################################################################################################
==============================NEW EXP: ASTE on cross_domain with seed 1398==============================
########################################################################################################
train_paths: ['incre/aste/cross_domain/total/train.txt']
dev_paths: ['incre/aste/cross_domain/total/dev.txt']
test_paths: ['incre/aste/cross_domain/total/test.txt']
train incre/aste/cross_domain/total/train.txt: 1205
train total: 1205
train count dict: {None: 1205}
aste/cross_domain/1114_0915-self-short/seed-1398/total-total/data/train.txt is written.
dev incre/aste/cross_domain/total/dev.txt: 171
dev total: 171
dev count dict: {'total': 1

Downloading:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/602 [00:00<?, ?B/s]

...

Gold: <neu> 장면 연출 <opinion> 만화같은 황당한
Gold list: [('장면 연출', '만화같은 황당한', 'neutral')]
Pred: <neg> 장면 연출 <opinion> 황당한
Pred list: [('장면 연출', '황당한', 'negative')]
Gold: <pos> 중견배우 <opinion> 노련한 <pos> 연기력 <opinion> 좋았던
Gold list: [('중견배우', '노련한', 'positive'), ('연기력', '좋았던', 'positive')]
Pred: <pos> 연기력 <opinion> 좋았던
Pred list: [('연기력', '좋았던', 'positive')]
Gold: <neg> 연출 <opinion> 제멋대로
Gold list: [('연출', '제멋대로', 'negative')]
Pred: <neg> 상황과 <opinion> 제멋대로
Pred list: [('상황과', '제멋대로', 'negative')]
Results of raw output
{'precision': 0.391812865497076, 'recall': 0.33004926108374383, 'f1': 0.3582887700534759}
Results of fixed output
{'precision': 0.391812865497076, 'recall': 0.33004926108374383, 'f1': 0.3582887700534759}
raw_scores
total
35.83
fixed_scores
total
35.83
Eval set count dict: {'total': 348}
Inferencing on test_aste ...


  0%|          | 0/87 [00:00<?, ?it/s]

raw_scores
total
36.91
fixed_scores
total
36.91
Eval set count dict: {None: 1205}
Inferencing on train_aste ...


  0%|          | 0/302 [00:00<?, ?it/s]

dev result over from epochs ['24']
raw_scores
lang	total
epoch-24[best]	39.18/33.00/35.83/	
fixed_scores
lang	total
epoch-24[best]	39.18/33.00/35.83/	
test result over from epochs ['24']
raw_scores
lang	total
epoch-24[best]	40.63/33.81/36.91/	
fixed_scores
lang	total
epoch-24[best]	40.63/33.81/36.91/	
The best checkpoint is aste/cross_domain/1114_0915-self-short/seed-1398/total-total/checkpoint-e24


In [ ]:
collate_seed_results(absa_config, runed_dirs)

Averaging constrained
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Avged 1 runs raw_scores
total-total	all
36.91	36.91
0.00	0.00
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Avged 1 runs fixed_scores
total-total	all
36.91	36.91
0.00	0.00
